# RabbitMQ

RabbitMQ é um intermediador de mensagens.  
1. Recebe e encaminha mensagens.  

Analogia com os correios: 
* Quando postamos uma carta, sabemos que a carta será entregue ao destinatário.
* Não nos preocupamos com o processo de entrega!

RabbitMQ é a caixa de correios, a agência e o carteiro!

## Componentes

### Producer (produtor)

Programa que **envia** as mensagens.  

![producer](https://s3-sa-east-1.amazonaws.com/lcpi/05c7260f-c02b-4d72-b748-d65d2aa9f0c6.png)

### Queue (fila)

* Equivalente à agência dos correios. 
* Mensagens só podem ser armazenadas dentro de filas.
* As filas são limitadas apenas pelas capacidades de disco e memória do servidor.
* Muitos produtores podem mandar mensagens para a mesma fila.
* Muitos consumidores podem consumir da mesma fila.

![queue](https://s3-sa-east-1.amazonaws.com/lcpi/c49ad77a-1ca8-42aa-a2ed-258c1a6086f2.png)

### Consumer (consumidor)

Programa que **espera para receber** as mensagens.

![consumer](https://s3-sa-east-1.amazonaws.com/lcpi/f274a1ff-0433-4ade-a33d-27c2da4e511d.png)

### ATENÇÃO

1. Producer, broker e consumer não precisam residir no mesmo servidor. 
2. Na prática, eles geralmente residem em servidores diferentes.
3. Uma aplicação pode funcionar como produtor e consumidor.

### Python library: pika 

```bash
    pip install pika --upgrade
```

In [1]:
!pip install pika --upgrade


[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Hello, RabbitMQ

Nossa primeira mensagem criada e envidada! 

## Enviando mensagens

![send_message](https://s3-sa-east-1.amazonaws.com/lcpi/cd48acd0-1db8-46cd-b769-8765dabfce00.png)

1. Estabelecendo uma conexão com o servidor RabbitMQ

```python
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
```


2. Criando a fila

```python
channel.queue_declare(queue='hello')
```


3. Publicando a mensagem

```python
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello RabbitMQ!'")
```

4. Garantindo a entrega para o servidor

```python
connection.close()
```

### Recebendo mensagens

![receiving](https://s3-sa-east-1.amazonaws.com/lcpi/bb976c70-c86a-4048-92b0-8ac3d182ee80.png)

1. Assegurando que a fila existe

```python
channel.queue_declare(queue='hello')
```

2. Recebendo mensagens: função *callback*  
   
   *Passo mais complexo, abstraído pela biblioteca **pika***.



```python
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
```

3. Indicar ao servidor qual função deve receber mensagens de uma fila específica

```python
channel.basic_consume(queue='hello',
                      auto_ack=True,
                      on_message_callback=callback)
```

4. Iniciar o processo de *espera para consumo*.

Loop infinito!

```python
print(' [*] Waiting for messages. To exit press CTRL+C')
channel.start_consuming()
```

5. Encerrar o loop

```python
if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        print('Interrupted')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)
```

# Tudo junto

![tudo_junto](https://s3-sa-east-1.amazonaws.com/lcpi/8db6cb00-42bf-41a2-b5b9-e8c0a2116bb0.png)

`01_producer.py`

```python
pass
```

`01_consumer.py`

```python
pass
```

# Como consumir desde o início da fila?


```python
channel.basic_consume(queue='hello', on_message_callback=callback, auto_ack=True, arguments={'x-stream-offset': 'first'})
```